In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.neural_network import MLPRegressor

# Load data into DataFrame and set index
df = pd.read_csv("D:/Projects/Forecasting/Sample.csv",parse_dates=['Date'], date_format = "%d-%m-%Y" )
df.set_index(df['Date'],inplace = True)
df.drop("Date",axis =1,inplace = True)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Projects/Forecasting/Sample.csv'

In [1]:
# Define functions for evaluation metrics
def mape(actual, forecast):
    return np.mean(np.abs((actual - forecast) / actual)) * 100

def wmape(actual, forecast):
    weights = actual / actual.sum()
    return np.sum(np.abs((actual - forecast) / actual) * weights) * 100
models = []
model_results = {}
final_results = pd.DataFrame()

# Loop through each item and generate forecasts
for item in range(0,len(df.columns)):
    series = df.iloc[:-4,item]

    # Decompose the time series (optional for seasonality analysis)
    decomposition = seasonal_decompose(series, model='additive')
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # Create models and generate forecasts
    
    # Holt-Winters
    models.append(ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=52,freq="W-MON").fit())  # Adjust seasonal_periods if needed
    model_results["Holt-Winters"] = models[0].forecast(steps=4)

    # ETS
    models.append(ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=52).fit())  # Adjust seasonal_periods if needed
    model_results["ETS"] = models[1].forecast(steps=4)

    # NNets
    models.append(MLPRegressor(hidden_layer_sizes=(5,), activation='relu', solver='adam').fit(series.index.values.reshape(-1, 1), series.values))
    model_results["NNET"] = models[2].predict(np.arange(len(series), len(series) + 4).reshape(-1, 1))

    # # ARIMA
    models.append(auto_arima(series, stepwise=True, trace=True))  # Find optimal ARIMA order
    model_results["ARIMA"] = models[3].predict(steps=4)

    # # SARIMA
    models.append(SARIMAX(series, seasonal=True).fit()) # Adjust seasonal_order if needed
    model_results["SARIMA"] = models[4].forecast(steps=4)

    # # Moving Average
    # models.append(series.rolling(window=5).mean().shift(1))
    # model_results["MVFORECAST"] = models.append(series.rolling(window=5).mean().shift(1))#models[5]#.iloc[-4:]

    mape_list = []
    wmape_list = []
    name = []
    # Evaluate model performance
    for model_name, forecast in model_results.items():
        mape_list.append(mape(df.iloc[-4:,item], forecast))
        wmape_list.append(wmape(df.iloc[-4:,item], forecast))
        name.append(model_name)
    
    final_results = pd.DataFrame({'Model':name,'MAPE': mape_list, 'WMAPE': wmape_list})
    print(final_results)
    # Select the best model based on MAPE and WMAPE
    
    final_results['Res'] = (final_results['MAPE'] + final_results['WMAPE']) / 2
    final_model = final_results[final_results['Res']==final_results['Res'].min()]['Model'].iloc[0]
    print(f"Best model for {item}: {final_model}")
    print(f"MAPE: {final_results[final_results['Model']==final_model]['MAPE'].iloc[0]:.2f}%")
    print(f"WMAPE: {final_results[final_results['Model']==final_model]['WMAPE'].iloc[0]:.2f}%")


    ## Final Forecasts
    # Generate and print the final forecast using the best model

NameError: name 'pd' is not defined

In [ ]:
models = ExponentialSmoothing(df['Item1'], trend='add', seasonal='add', seasonal_periods=52).fit()  # Adjust seasonal_periods if needed
models.forecast(steps=4)

df.iloc[-4:,item]
forecast
df.iloc[-4:,item]-forecast